# Understanding the MCPFromScratch Project: A Beginner's Guide

## 1. Introduction to MCPFromScratch
Welcome! This notebook will walk you through the `MCPFromScratch` project. It's designed to show how a **client** (like a smart AI agent) can talk to a **server** that offers various services. These services might be 'tools' (like a calculator or a database searcher), 'prompts' (like a text summarizer), or 'resources' (like user profiles).The project uses modern Python technologies like FastAPI for building the server and WebSockets for real-time communication. A key concept here is the **Model Context Protocol (MCP)**, which is a special set of rules defining how the client and server exchange messages. Let's dive in!

## 2. Core Concepts for Beginners

### Client-Server ExplainedImagine a restaurant:*   **Server:** Like the kitchen and staff. It has resources (ingredients, chefs) and provides services (cooking food, taking orders). It waits for customers (clients) to make requests.*   **Client:** Like you, the customer. You make requests (order food) and receive responses (your meal).In our project:*   The **Server** (`MCPFromScratch/server/`) is a Python program that offers tools, prompts, and resources. It listens for requests over the internet.*   The **Client** (`MCPFromScratch/client/`) is another Python program (an 'agent') that connects to the server, asks it to do things (e.g., 'list available tools' or 'run this tool'), and gets results back.

### What is the Model Context Protocol (MCP)?MCP stands for **Model Context Protocol**. It's **not** a standard internet protocol like HTTP, but a **custom-designed set of rules** for this specific project. Think of it as a special language that the client and server agree to use so they can understand each other perfectly. The 'Model' part hints that this protocol is particularly useful when AI models (like Large Language Models - LLMs) are involved, perhaps on the client-side to decide what to ask the server, or on the server-side to provide intelligent services.**Purpose:** To define a clear, structured way for the client and server to communicate about:   - What capabilities (tools, prompts, resources) the server has.   - How to ask the server to use one of these capabilities.   - How the server sends back results or errors. This structured communication is defined in the `protocol_types.py` files found in both the `server` and `client` directories. These files list all the valid message types and what information each message should contain.

### FastAPI: Building the ServerFastAPI is a modern, fast (high-performance) web framework for building APIs (Application Programming Interfaces) with Python. In this project, it's used to build the server-side application. It helps handle incoming web requests, route them to the correct Python functions, and send back responses. It's known for being easy to use and automatically generating documentation for your API.

### WebSockets: Real-time ChatImagine a phone call versus sending letters. HTTP (the standard web protocol) is like sending letters – you send a request, you get a response, and the connection closes. WebSockets are like a phone call – once the connection is made, it stays open, allowing both the client and server to send messages to each other at any time. This is great for real-time applications, like a chat app, or our MCP agent that needs continuous interaction with the server. The `MCPFromScratch` project uses WebSockets for its main communication channel (`/mcp` endpoint).

### Pydantic: Keeping Data TidyPydantic is a Python library for data validation and settings management using Python type hints. Whenever the client sends data to the server, or the server sends data back, Pydantic helps ensure the data is in the correct format. If you expect a number but get text, Pydantic will complain! This helps catch errors early and makes the code more robust. You'll see Pydantic models used extensively in the `protocol_types.py` files to define the structure of messages.

### Async/Await: Doing Many Things at OnceModern web servers need to handle many client requests simultaneously. `async` and `await` are Python keywords that allow programs to perform non-blocking operations. This means if one part of the code is waiting for something (like a database query or a network response), other parts can continue running. This makes the server efficient and responsive. You'll see `async def` for defining asynchronous functions throughout the codebase.

## 3. The Server-Side (Where the Magic Happens)

The server is responsible for providing the actual services (tools, prompts, resources). Let's look at its key components located in the `MCPFromScratch/server/` directory.

### `main.py`: The Starting PointThis file is the main entry point for the server application.*   **FastAPI App Setup:** It initializes the `MCPWizServer` (which itself sets up a FastAPI application). `mcp = MCPWizServer(\My App\, lifespan=app_lifespan)`*   **Lifespan Management (`app_lifespan`):** This function manages resources that should be available throughout the application's life, like connecting to and disconnecting from a database (`Database(\sqlite+aiosqlite:///./demo.db\)). It also sets up an `InMemoryKeyStore` for API key authentication.*   **Defining Tools (`@mcp.tool()`):** Functions like `text_to_sql_query`, `query_db`, and `add` are decorated with `@mcp.tool()`. This registers them with the server as callable tools. The `text_to_sql_query` tool is interesting as it uses an OpenAI client to convert natural language to SQL.*   **Defining Prompts (`@mcp.prompt()`):** Functions like `summarize_text` and `translate_text` are decorated with `@mcp.prompt()`. These define templates that can be filled with arguments to generate prompts for an LLM.*   **Defining Resources (`@mcp.resource()`):** Functions like `get_config` and `get_user_profile` are decorated with `@mcp.resource()`. These make data or services available via a URI-like pattern (e.g., `users://{user_id}/profile`).*   **Environment Variables:** It uses `load_dotenv()` to load configuration like API keys from a `.env` file.*   **Running the Server:** The `if __name__ == \__main__\:` block shows how the server can be started directly using `mcp.run()` (though `uvicorn` is typically used for development/production).

### `server.py` (`MCPWizServer`): The Conductor of the OrchestraThis file defines the `MCPWizServer` class, which is the core framework for our MCP server.*   **Initialization (`__init__`):** Sets up the FastAPI application and internal 'registries' (think of them as lists) to keep track of all available tools, prompts, and resources.*   **Decorators (`@tool`, `@prompt`, `@resource`):** These are special functions that wrap other functions. When you use `@mcp.tool()` on your function, this decorator adds your function to the server's list of tools, automatically figures out what inputs it needs (its 'schema') from your Python type hints, and makes it available via the API.*   **Setting up Routes (`_setup_routes`):** This is crucial. It defines all the web addresses (endpoints) that the server will respond to. This includes:    *   **HTTP Endpoints:** Standard web requests like `/health` (to check if the server is running), `/info` (to get server details), `/tools/list` (to get a list of tools), and `/tools/call` (to execute a tool). Many of these require API key authentication (`Depends(api_key_auth)`).    *   **WebSocket Endpoint (`/mcp`):** This is the main endpoint for real-time communication using the Model Context Protocol. When a client connects here, it sets up a persistent session.*   **Handling HTTP Requests:** Methods like `_invoke_tool_http`, `_invoke_prompt_http`, and `_invoke_resource_http` contain the logic for what to do when an HTTP request comes in for a specific tool, prompt, or resource.*   **WebSocket Connection Logic (`mcp_endpoint`):**     *   Authenticates the client using an `X-API-Key` header and the `InMemoryKeyStore`. It checks if the key is valid and if the user has quota available.    *   Accepts the WebSocket connection.    *   Creates a `WizServerSession` object to manage this specific client's conversation.    *   Uses `anyio` to run multiple tasks concurrently: reading messages from the client, writing messages to the client, and the session's own processing loop.

### `server_session.py` (`WizServerSession`) & `base_session.py` (`ServerSession`): Managing ConversationsThese files handle the logic for an individual client's session once they've connected via WebSocket.**`base_session.py` (`ServerSession`):***   **`RequestResponder` Class:** A helper to make it easy to send a success response or an error response back to the client for a specific request.*   **`ServerSession` Class:** The base class for WebSocket sessions.    *   `__init__`: Stores references to the communication streams, initialization options, API key, key store, and application state.    *   `_receive_loop()`: This is a core loop.         1.  It first sends an `InitializationResponse` to the client, telling it about the server's capabilities.        2.  Then, it continuously listens for messages from the client.        3.  For requests that should be rate-limited (like tool calls), and if authentication is enabled, it checks the client's API key against the `key_store` and validates/increments their usage quota. If the quota is exceeded or the key is invalid, it sends an error and skips processing.        4.  Valid client requests are wrapped in a `RequestResponder` and put into a queue (`self.incoming_messages`) for further processing.**`server_session.py` (`WizServerSession`):***   This class inherits from `ServerSession` and adds more specific logic for the MCPWiz server.*   `__init__`: Stores the tool, prompt, and resource registries.*   `start_dispatching()`: An asynchronous loop that takes requests from the `incoming_messages` queue (populated by `_receive_loop` in the base class) and calls `_handle_request` for each one.*   `_handle_request()`: This method uses a `match` statement (modern Python's version of a switch/case) to determine the type of request (e.g., `ToolCallRequest`, `PromptsListRequest`) and calls the appropriate internal method to handle it (e.g., `_exec_tool`, `_exec_prompt`). It also handles generic requests like `/health` and `/info` if they come over WebSocket.    *   For `ToolsListRequest`, it demonstrates sending `ProcessUpdate` messages to the client to indicate progress before sending the final list.*   `_exec_tool()`, `_exec_prompt()`, `_exec_resource()`: These methods contain the logic to:    1.  Find the requested tool/prompt/resource in the server's registry.    2.  Validate the arguments provided by the client against the expected schema.    3.  If the tool/prompt/resource expects a `Context` object (for accessing shared application state like the database), it creates and passes one.    4.  Execute the actual handler function for the tool/prompt/resource.    5.  Send the result (or an error) back to the client using the `RequestResponder`.

### `protocol_types.py` (Server-side): The Server's LanguageThis file is crucial. It defines all the data structures (using Pydantic models) for messages exchanged between the client and server according to the Model Context Protocol (MCP). It's like the official dictionary and grammar book for their communication.It includes definitions for:*   **Requests:** What the client can ask for (e.g., `ToolCallRequest`, `ToolsListRequest`, `PromptCallRequest`, `ResourceCallRequest`, `HealthRequest`, `InfoRequest`). Notice the `method` field often acts as a discriminator to tell different request types apart.*   **Results:** What the server sends back on success (e.g., `ToolCallResult`, `ToolsListResult`).*   **Errors:** `ServerError` for when things go wrong.*   **Capabilities:** How tools, prompts, and resources are described (`ToolCapability`, `ServerCapabilities`, `_BareToolSpec`, `_BarePromptSpec`, `_BareResourceSpec`).*   **Initialization:** `InitializationRequest` (from client), `InitializationResponse` (from server), `InitializationOptions` (server configuration for a session).*   **WebSocket Framing:** `SessionMessage` is the top-level wrapper for *all* messages sent over the WebSocket. It has a `root` field that contains the actual specific message (like a `ToolCallRequest` or a `ServerError`).*   `ClientRequest` and `ServerResult` are `Union` types, meaning a message can be one of several defined types. Pydantic uses the `method` field in requests to figure out which specific model to use for parsing.

### `context_models.py`: Giving Tools Access to Important InfoThis file defines how tools, prompts, or resources can get access to application-wide information (like a database connection) or request-specific details.*   `LifespanContext`: Wraps objects made available during the application's lifespan (e.g., `app.state` which holds the database and key store in `main.py`).*   `RequestContext`: Holds information about the current request (e.g., the FastAPI `Request` object itself, though this is `None` for WebSocket calls) and the `LifespanContext`.*   `Context`: The main object that gets injected into tool/prompt/resource functions if they declare `ctx: Context` as their first parameter. It contains the `RequestContext`.

### Special Features: Tools, Prompts, and ResourcesThe server offers three main types of capabilities:*   **Tools:** These are functions that perform specific actions. Examples: `add(a, b)` for addition, `text_to_sql_query(text)` to convert natural language to a SQL query and execute it. The client can ask the server to run a tool with specific arguments.*   **Prompts:** These are templates for generating text, typically for an LLM. Examples: `summarize_text(text_to_summarize)` which creates a prompt to ask an LLM to summarize text. The client provides arguments, the server uses the registered prompt function to format these into a larger text prompt, and then returns this formatted prompt string to the client (which would then presumably send it to an LLM). *   **Resources:** These represent data or services that can be accessed via a URI-like pattern. Examples: `config://app` for static app configuration, `users://{user_id}/profile` for dynamic user data. The server matches the requested URI to a registered handler and executes it.

### Security: API Keys and QuotasThe server implements security measures:*   **API Key Authentication:** Most HTTP endpoints and the WebSocket endpoint require an API key (`X-API-Key` header for WebSocket, or as part of the request for HTTP). This ensures that only authorized clients can access the server.*   **Quota Management:** The `InMemoryKeyStore` (defined in `utilities/auth.py`, though the file itself is not provided in the context, its usage is clear) is used to store API keys and track their usage against a predefined quota. This prevents abuse and ensures fair usage. The `ServerSession` checks and increments usage for rate-limited requests.

## 4. The Client-Side (The User's Gateway)

The client is responsible for interacting with the user (or acting on its own if it's an autonomous agent) and communicating with the server to get things done. The client code is in `MCPFromScratch/client/`.

### `main_agent.py` (`ChatAssistant`): The Smart MessengerThis file defines the `ChatAssistant` class, which acts as an intelligent agent.*   **Initialization (`__init__`):** Takes the server's WebSocket URI, the MCP API key (for the server), and an OpenAI API key (for its own LLM usage).*   **Connection (`connect` & `_initialize_session`):** Establishes a WebSocket connection to the server, sending the required `X-API-Key` header. It then performs the MCP initialization handshake by sending an `InitializationRequest` and expecting an `InitializationResponse` back from the server.*   **Listing Tools (`_call_tools_list`):** Sends a `ToolsListRequest` to the server and receives a `ToolsListResult` containing details of available tools. It also handles intermediate `ProcessUpdate` messages from the server, printing them to the console.*   **Calling a Tool (`_call_tool`):** Sends a `ToolCallRequest` (with tool name and arguments) to the server and expects a `ToolCallResult` or a `ServerError` back.*   **Handling User Queries (`handle_user_query`):** This is the main logic flow for the agent:    1.  It first gets the list of available tools from the server (`_call_tools_list`).    2.  It then uses an LLM (OpenAI's GPT-4o, via `_select_tool_with_llm`) to:         a.  Understand the user's natural language query.         b.  Choose the most appropriate tool from the server's list.         c.  Determine the correct arguments for that tool based on its schema.    3.  It executes the chosen tool by calling the server (`_call_tool`).    4.  Finally, it takes the (often raw, JSON-like) result from the tool and uses the LLM again (`_generate_conversational_answer_with_llm`) to formulate a friendly, natural language answer for the user.*   **LLM Integration (`_select_tool_with_llm`, `_generate_conversational_answer_with_llm`):** These methods show how to construct prompts for an LLM. For tool selection, it provides the LLM with the user's query and a detailed description of available tools (including their input schemas). For generating a conversational answer, it gives the LLM the original query and the tool's raw output.*   **Closing (`close`):** Closes the WebSocket connection.*   **Example Usage (`async def main()`):** Demonstrates how to create an assistant, connect, handle a query, print the answer, and close the connection. It uses `asyncio.run()` to execute the asynchronous code.

### `protocol_types.py` (Client-side): The Client's LanguageSimilar to the server's version, this file defines the Pydantic models for messages that the client sends or receives. It's essential for the client to correctly structure its requests and understand the server's responses. It's often a subset of the server's `protocol_types.py`, containing only the message types relevant to client-server WebSocket communication for this specific agent's needs (e.g., it might omit types only used for HTTP or server-internal options not directly exchanged).

## 5. Understanding the Model Context Protocol (MCP) in Depth

As mentioned, MCP is a custom protocol for this application. Let's reiterate its purpose and key aspects:**Purpose:** The primary goal of MCP is to create a standardized way for a client (especially one driven by an AI model) to discover and use capabilities (tools, prompts, resources) offered by a server. It aims to make the server's functions easily consumable by an intelligent agent.**Design Principles (Inferred):***   **Explicit Handshake:** The `InitializationRequest` / `InitializationResponse` ensures both client and server know they are starting an MCP session and can exchange capabilities.*   **Structured Requests/Responses:** Using Pydantic models (defined in `protocol_types.py`) for all messages ensures clarity and reduces errors. Data is validated against these models.*   **Capability Discovery:** Endpoints like `tools/list`, `prompts/list`, `resources/list` allow the client to dynamically find out what the server can do.*   **Discriminated Unions:** The `method` field in requests and `type` field in some results (like `ToolCallResult`) help in distinguishing between different kinds of messages or payloads within a general structure.*   **Error Handling:** `ServerError` provides a standard way to communicate issues.*   **Session-Based:** Communication primarily happens over a persistent WebSocket connection, managed as a session.*   **Extensibility:** The protocol can be extended by adding new request/result types for new capabilities.**Key Message Types Revisited:***   `SessionMessage`: The outer envelope for all WebSocket messages. Contains a `root` field holding the actual payload.*   `InitializationRequest` & `InitializationResponse`: For starting the session and exchanging server capabilities.*   `ToolsListRequest` & `ToolsListResult`: For the client to get a list of available tools with their descriptions and input schemas.*   `ToolCallRequest` & `ToolCallResult`: For the client to ask the server to execute a specific tool with given arguments, and for the server to return the result.*   `PromptsListRequest` & `PromptsListResult`: Similar to tools, but for listing available prompt templates.*   `PromptCallRequest` & `PromptCallResult`: For asking the server to generate a prompt string using a named template and arguments.*   `ResourcesListRequest` & `ResourcesListResult`: For listing available resources.*   `ResourceCallRequest` & `ResourceCallResult`: For fetching a resource by its URI.*   `ProcessUpdate`: Allows the server to send intermediate progress messages to the client during a long-running operation (e.g., while preparing the tools list).*   `ServerError`: For the server to report errors to the client.*   `CloseSession`: For the server to indicate it's closing the session.Essentially, MCP provides the vocabulary and grammar for the client and server to have a productive conversation about using the server's special abilities.

## 6. Setting Up and Running the Application

Here’s how you can get the `MCPFromScratch` application running on your own machine.

### Prerequisites*   Python 3 (typically 3.8+ for FastAPI and modern type hinting features).*   `pip` (Python package installer).

### Installation StepsThese instructions primarily cover the server setup. You'll need a similar environment for the client if it has its own dependencies (though for this project, client dependencies are likely covered by the server's `requirements.txt` if run in the same general project setup, or would be minimal). Assume you have cloned the project and are in its root directory.1.  **Navigate to the server directory:**    ```bash    cd MCPFromScratch/server    ```2.  **Create and activate a virtual environment:** This is highly recommended to keep project dependencies separate.    *   **Linux/macOS:**        ```bash        python3 -m venv venv        source venv/bin/activate        ```    *   **Windows (PowerShell):**        ```bash        python -m venv venv        .\\venv\\Scripts\\Activate.ps1        ```3.  **Install dependencies:** The `requirements.txt` file (expected to be in the `MCPFromScratch/server/` directory) lists all necessary Python packages.    ```bash    pip install -r requirements.txt    ```    *(Note: If the client has a separate `requirements.txt` in `MCPFromScratch/client/`, you'd repeat steps 2 & 3 for the client in its directory, or ensure all dependencies are in a shared `requirements.txt` at the project root if using a single environment.)*

### Environment Variables (API Keys)Both the server (`main.py`) and client (`main_agent.py`) use `load_dotenv()` to load environment variables from a `.env` file. You'll need to create this file in the respective directories (or a common root if configured that way).**For the Server (`MCPFromScratch/server/.env`):**   ```env   OPENAI_API_KEY=your_openai_api_key_here  # If tools use OpenAI, like text_to_sql_query   # Other server-specific variables if any   ```**For the Client (`MCPFromScratch/client/.env`):**   ```env   MCP_CLIENT_API_KEY=your_chosen_api_key_for_mcp_server  # This key must be known by the MCP server's InMemoryKeyStore   OPENAI_API_KEY=your_openai_api_key_here # For the client agent's own LLM use   ```The `MCP_CLIENT_API_KEY` is what the client sends as `X-API-Key`. The server's `InMemoryKeyStore` (in `server/main.py`'s `app_lifespan` or `utilities/auth.py`) needs to be configured to recognize this key and its associated quota. For this example, it seems the `InMemoryKeyStore` might be pre-populated or would need a mechanism to add keys (not detailed in provided code, but implied by its usage).

### Running the ServerFrom the `MCPFromScratch/server/` directory (with the virtual environment activated):```bashuvicorn main:mcp.app --reload --port 8000```*   `uvicorn`: An ASGI server, necessary to run FastAPI applications.*   `main:mcp.app`: Tells Uvicorn to find the FastAPI app instance named `app` (which is an attribute of the `mcp` object, an instance of `MCPWizServer`) inside the `main.py` file.*   `--reload`: Enables hot reloading, so the server restarts automatically when you save code changes (great for development).*   `--port 8000`: Runs the server on port 8000. The client will connect to `ws://localhost:8000/mcp`.

### Running the ClientFrom the `MCPFromScratch/client/` directory (with its virtual environment activated, if separate, and `.env` file present):```bashpython main_agent.py```This will execute the `main()` function in `main_agent.py`, which connects to the server, sends a query, prints the answer, and then closes.

## 7. Conclusion

Congratulations! You've explored the `MCPFromScratch` project. You've learned about:*   The **Client-Server** model.*   The custom **Model Context Protocol (MCP)** and its message structures.*   How the **server** is built using FastAPI, offering tools, prompts, and resources.*   How the **client agent** uses WebSockets to communicate with the server and an LLM to make intelligent decisions.*   Key technologies like Pydantic for data validation and async/await for concurrency.*   How to set up and run the application.This project provides a solid foundation for understanding how to build sophisticated applications where AI agents can interact with powerful, well-defined server capabilities. You can experiment by adding new tools, prompts, or resources to the server, or by enhancing the client agent's logic!